In [1]:
## Import relevant libraries
import sys
import glob

sys.path.append('../../py_files/')
import quadrop2 as qd

qd.set_plotting_style()

### Data pre-procesing

In [2]:
# Example usage
base_dir = "../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/021625-HeAl-titrations-RT/"
qd.consolidate_images(base_dir)

Consolidation appears to be already done. Directory '../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/021625-HeAl-titrations-RT/2p5ulTMB-1ulDNA_' already exists with subfolders.


In [4]:
# Example usage
data_path = "../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/021625-HeAl-titrations-RT/2p5ulTMB-1ulDNA_/"

calibration_curve_paths = sorted(glob.glob("../../../../Thomson Lab Dropbox/David Larios/activedrops/calibration_curve/***ugml.tif"))


conditions_dict = {
    "HeAl_160nM-RT": "Pos0", 
    "HeAl_80nM-RT": "Pos1", 
    "HeAl_40nM-RT": "Pos2", 
    "HeAl_20nM-RT": "Pos3", 
    "HeAl_10nM-RT": "Pos4", 
    "HeAl_5nM-RT": "Pos5", 
    "HeAl_2p5nM-RT": "Pos6", 
    "HeAl_1p25nM-RT": "Pos7", 
}

# Organize PosX folders into condition folders
qd.organize_conditions(data_path, conditions_dict)

# Now run the existing functions to reorganize the tiffs and rename the folders
conditions, subconditions = qd.prepare_conditions(data_path)
time_interval_list = [6] * len(conditions)  # time intervals in seconds between frames for each condition
subconditions = ['Rep1']

print("Conditions:", conditions)
print("Subconditions:", subconditions)



Conditions: ['HeAl_1p25nM-RT', 'HeAl_2p5nM-RT', 'HeAl_5nM-RT', 'HeAl_10nM-RT', 'HeAl_20nM-RT', 'HeAl_40nM-RT', 'HeAl_80nM-RT', 'HeAl_160nM-RT']
Subconditions: ['Rep1']


In [ ]:
qd.reorgTiffsToOriginal(data_path, conditions, subconditions)


### Generate movies

In [ ]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions[:], 
    subconditions, 
    channel='cy5', 
    time_interval_list=time_interval_list, 
    vmax=14, 
    skip_frames=4, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=False,
    )

In [ ]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    frame_rate=60,
    skip_frames=1
    )


In [ ]:
conditions[::-1]

In [ ]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions[::-1], 
    subconditions, 
    channel='cy5', 
    grid_rows=2, 
    grid_cols=4, 
    frame_rate=60,
    batch_size=50
    )


In [5]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    time_interval_list=time_interval_list, 
    vmax=400, 
    skip_frames=4, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=True,
    )

Processing HeAl_160nM-RT - Rep1: 100%|██████████| 1000/1000 [01:44<00:00,  9.60it/s]


In [ ]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    frame_rate=60,
    skip_frames=1
    )


In [6]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions[::-1], 
    subconditions, 
    channel='GFP', 
    grid_rows=2, 
    grid_cols=4, 
    frame_rate=60,
    batch_size=50
    )


Creating video with duration: 16.67 seconds.


Creating combined frames: 100%|██████████| 1000/1000 [04:50<00:00,  3.44it/s]


Combined video saved to ../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/021625-HeAl-titrations-RT/2p5ulTMB-1ulDNA_/output_data/movies/combined_heatmap_movie_GFP_60fps_1000frames.avi


In [ ]:
# qd.delete_temporary_image_directories(data_path, conditions, subconditions)

### Fluorescence Quantification

In [ ]:
conditions

In [ ]:
# Example usage
# mw_kda_list = [44.95] * len(conditions)
droplet_volume_list = [2] * len(conditions)
# protein_lengths_list = [401] * len(conditions) # last one is negative

qd.quantify_tiffiles(
    data_path, 
    conditions[:], 
    subconditions, 
    calibration_curve_paths, 
    droplet_volume_list, 
    time_interval_list, 
    skip_frames=10,

)

### PIV pre-processing

In [ ]:
conditions[1:2:]

In [ ]:
qd.split_tiffs(data_path, conditions[1:2:], subconditions, channel='cy5', file_interval=1)



### PIV

In [ ]:
# Define feature limits and other parameters
v = 2E-7
velocity_limits = (0, v)
other_limits = (-0.0005, 0.0005)
skip_frames = 1 ### CHANGE THIS TO SKIP FRAMES


velocity_limits = (None, None)
other_limits = (None, None)


feature_limits = {
    # 'u [m/s]': (-v, v), 
    # 'v [m/s]': (-v, v), 
    # 'data type [-]': (None, None),
    'velocity magnitude [m/s]': velocity_limits,
    'vorticity [1/s]': other_limits,
    'divergence [1/s]': other_limits,
    # 'dcev [1]': (0, 250),
    'shear [1/s]': other_limits,
    'strain [1/s]': other_limits,
    'vector direction [degrees]': (-180, 180),
}


# Features for PCA and plotting
features_pca = [
    "vorticity [1/s]_mean",
    "velocity magnitude [um/s]",
    "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    "correlation length [um]", 
    "power [W]_mean",
    "work [J]",
]


In [ ]:
time_interval_list = [6] * len(conditions)  # time intervals in seconds between frames for each condition


# Process PIV data
qd.process_piv_data(
    data_path, 
    conditions,    
    subconditions, 
    feature_limits, 
    time_interval_list, 
    min_frame=0, 
    max_frame=None, 
    skip_frames=1, 
    plot_autocorrelation=False, 
    frame_rate=1, 
    heatmaps=False
    )


In [ ]:
# Plot features and PCA
qd.plot_PIV_all(
    data_path, 
    conditions[:5:-1],
    subconditions, 
    features_pca, 
    min_frame=0, 
    max_frame=None
    )


### Expression + PIV

In [ ]:
# Combine the data and save it to the "output_data" directory
qd.combine_averaged_dataframes(data_path, conditions, subconditions)


In [ ]:
qd.merge_expression_piv_data(data_path)


In [ ]:
# x_column = "time (h)"  # Example x-axis column
# y_column = "Protein Concentration_nM"  # Example y-axis column


# x_column = "Protein Concentration_nM"  # Example y-axis column
# y_column = "velocity magnitude [m/s]_mean"  # Example y-axis column

# x_column = "time (h)"  # Example x-axis column
# y_column = "velocity magnitude [m/s]_mean"  # Example y-axis column

# x_column = "time (h)"  # Example x-axis column
# y_column = "power [W]_mean"  # Example y-axis column

# x_column = "time (h)"  # Example x-axis column
# y_column = "work [J]_mean"  # Example y-axis column

x_column = "time (h)"  # Example x-axis column
y_column = "distance [m]_mean"  # Example y-axis column


qd.plot_expression_piv(
    data_path,
    conditions,
    x_column, 
    y_column, 
    sigma_x=0.1, 
    sigma_y=10, 
    x_log=False, 
    y_log=True, 
    min_frame=0, 
    max_frame=None, 
    individual_plots=False
    )

In [ ]:

# List of features for PCA
features_pca = [
    "vorticity [1/s]_mean",
    "velocity magnitude [m/s]_mean",
    "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    "correlation length [m]_mean", 
    "power [W]_mean",
    "work [J]_mean",
    'vector direction [degrees]_mean',
    "Protein Concentration_nM", 
    'time (min)'
]

# Run PCA and save plot (with all conditions and subconditions in the same plot)
qd.plot_pca_expression_piv(data_path, conditions=conditions, subconditions=subconditions, features=features_pca, sigma=1)


In [ ]:

# Example usage
qd.delete_outputs(data_path, conditions, subconditions, output_dirs=None)
